<a href="https://colab.research.google.com/github/sccn/sound2meg/blob/main/wav2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install fairseq

In [ ]:
pip install transformers

In [3]:
import re
import numpy as np
#import torch
import os
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2FeatureExtractor
import librosa
import fairseq
import torch
import pandas as pd

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [4]:
def segment_audio(audio_file):
    audio_input, sample_rate = librosa.load(audio_file)
    audio_segment = sample_rate * 3
    segments = []
    if len(audio_input) >= audio_segment * 2:
      check = True
      while check:
          segments.append(audio_input[:audio_segment])
          audio_input = audio_input[audio_segment:]
          if len(audio_input) < audio_segment:
              check = False
    else:
      segments.append(audio_input[:audio_segment])
    return segments, sample_rate

# return first 3 second segment in audio file
def three_second(audio_file):
    audio_input, sample_rate = librosa.load(audio_file)
    audio_segment = sample_rate * 3

    segment = audio_input[:audio_segment]
    return segment, sample_rate


def read_event(tsv_path):
    event = pd.read_csv("sub-A2014_task-auditory_events.tsv", sep = '\t')
    event = event[event['type'] == 'Sound']

    batch["onset"], batch["sample"], batch["audio_file"] = event['onset'], event['sample'], event['audio_file']
    
    return batch


def prepare_dataset(batch, feature_extractor):
    for audio in audio_file_directory:
        if audio in batch["audio_file"]:
            segment, sample_rate = three_second(audio)
            #segment, sample_rate = segment_audio(audio)
            
            feature_extractor = prepare_feature_extractor(sample_rate)
            batch["input_values"] = feature_extractor(segment, return_tensors="pt").input_values

    return batch

def prepare_model(model_path):
    model = Wav2Vec2ForCTC.from_pretrained(model_path)

    return model

def prepare_feature_extractor(sampling_rate):
    # check that all files have the correct sampling rate
    feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=sampling_rate, padding_value=0.0, do_normalize=True, return_attention_mask=True)

    return feature_extractor

In [5]:
model_path = "facebook/wav2vec2-large-xlsr-53"
model = prepare_model(model_path)

Downloading:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['project_q.bias', 'project_q.weight', 'quantizer.weight_proj.weight', 'project_hid.bias', 'quantizer.codevectors', 'quantizer.weight_proj.bias', 'project_hid.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to u

In [6]:
model

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (2): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (3): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elemen

In [8]:
np.load('907.npy').shape

(1, 206, 32)

In [ ]:
import re
import numpy as np
import torch
import os
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2FeatureExtractor
import librosa
import fairseq
import torch
import pandas as pd


model_path = "facebook/wav2vec2-large-xlsr-53"
audio_path = "/content/audio"

model = prepare_model(model_path)
files = os.listdir(audio_path)

# for wav_file in files:
#     count = 0
#     name = re.search('(\d+)\.', wav_file).group().replace('.', '_')

#     audio = os.path.join(audio_path, wav_file)

#     audio_input, sample_rate = segment_audio(audio)
#     feature_extractor = prepare_feature_extractor(sample_rate)

#     if len(audio_input) > 1:
#         for audio in audio_input:
#             input_values = feature_extractor(audio, return_tensors="pt").input_values
#             logits = model(input_values).logits

#             output = logits.detach().numpy()
#             np.save(f'{name}{count}.npy', output)

#             count += 1
#     else:
#         input_values = feature_extractor(audio_input, return_tensors="pt").input_values
#         logits = model(input_values).logits

#         output = logits.detach().numpy()
#         np.save(f'{name}.npy', output)

# Only first three second segements
for wav_file in files:
    print(wav_file)
    name = re.search('(\d+)\.', wav_file).group().replace('.', '')

    audio = os.path.join(audio_path, wav_file)

    audio_input, sample_rate = three_second(audio)
    feature_extractor = prepare_feature_extractor(sample_rate)

    input_values = feature_extractor(audio_input, return_tensors="pt").input_values
    logits = model(input_values).logits

    output = logits.detach().numpy()
    np.save(os.path.join('/content/Embedding/MyDrive/Embedding', f'{name}.npy'), output)
